#### <b>Поиск паттернов и обход деревьев зависимостей </b>
В этой главе рассматриваются еще два подхода: использование паттернов (закономерностей) последовательностей слов для классификации и генерации текста, а также выделение из высказывания необходимых элементов информации путем обхода его дерева синтаксических зависимостей. 


Вы познакомитесь с утилитой Matcher библиотеки spaCy, предназначенной для поиска закономерностей.

А также узнаете, в каких случаях для определения нужного подхода к обработке все равно приходится учитывать контекст

In [1]:
import ru_core_news_lg
import spacy as sp
import sys

from spacy.symbols import ORTH, LEMMA
from IPython.display import Image
from spacy.tokens.doc import Doc
from spacy.vocab import Vocab

from spacy import displacy
from IPython.display import Image

from function import *

nlp = sp.load('ru_core_news_lg')


In [2]:
print('{:^30} {:<30}'.format('left aligned', "dfsd"))

         left aligned          dfsd                          


#### Паттерны последовательностей слов
Паттерн последовательности слов (word sequence pattern) :  I can -> «местоимение + вспомогательный модальный глагол»

In [3]:
doc1 = nlp(u'Мы можем догнать их.')
doc2 = nlp(u'Вы должны указать это.')

print('{:<10} {:<10} {:<10} {:<10}'.format("doc1", "doc2", "dep_", "explain"))
print("-" * 45)
for i in range(len(doc1)-1):
    if doc1[i].dep_ == doc2[i].dep_:
        print('{:<10} {:<10} {:<10} {:<10}'.format(doc1[i].text, doc2[i].text, doc1[i].dep_, sp.explain(doc1[i].dep_)))

doc1       doc2       dep_       explain   
---------------------------------------------
Мы         Вы         nsubj      nominal subject
можем      должны     ROOT       root      
догнать    указать    xcomp      open clausal complement
их         это        obj        object    


#### <b>Задание</b>
Перепишите сценарий таким образом, чтобы создавался один общий объект Doc.

In [4]:
doc = nlp(u"Мы можем догнать их. Вы должны указать это. Я хочу летать там. Они пошли учить что-то. Настя любит учить уроки. Птица улетела на юг.")

sents = list(doc.sents)
count = sum([+1 for token in sents[0]])

for s1 in sents:
    for s2 in sents:
        if (s1 != s2):
            for i in range(0, count - 1):
                if (s1[i].dep_ == s2[i].dep_):
                    print('{:<10} {:<10} {:<10} {:<10}'.format(s1[i].text, s2[i].text, s1[i].dep_, sp.explain(s1[i].dep_)))
            print("-" * 50)

Мы         Вы         nsubj      nominal subject
можем      должны     ROOT       root      
догнать    указать    xcomp      open clausal complement
их         это        obj        object    
--------------------------------------------------
Мы         Я          nsubj      nominal subject
можем      хочу       ROOT       root      
догнать    летать     xcomp      open clausal complement
--------------------------------------------------
Мы         Они        nsubj      nominal subject
можем      пошли      ROOT       root      
догнать    учить      xcomp      open clausal complement
их         что        obj        object    
--------------------------------------------------
Мы         Настя      nsubj      nominal subject
можем      любит      ROOT       root      
догнать    учить      xcomp      open clausal complement
их         уроки      obj        object    
--------------------------------------------------
Мы         Птица      nsubj      nominal subject
можем      улет

#### Проверка высказывания на соответствие паттерну
Для разделения различных типов высказываний можно проверить, соответствуют ли высказывания следующему паттерну: «подлежащее + вспомогательный глагол + глагол + ... + прямое дополнение...».
 
Пропуски указывают, что прямое дополнение не обязано располагаться сразу за глаголом,

In [5]:
doc = nlp(u"Солнце зашло три часа назад, небо стало выглядеть очень мрачно ночью.")
nice_print(doc)

text       pos_       dep_         tag_      
_______________________________________________________
Солнце     PROPN      nsubj        PROPN     
зашло      VERB       ROOT         VERB      
три        NUM        nummod:gov   NUM       
часа       NOUN       obl          NOUN      
назад      ADV        advmod       ADV       
,          PUNCT      punct        PUNCT     
небо       NOUN       nsubj        NOUN      
стало      VERB       conj         VERB      
выглядеть  VERB       xcomp        VERB      
очень      ADV        advmod       ADV       
мрачно     ADV        advmod       ADV       
ночью      NOUN       obl          NOUN      
.          PUNCT      punct        PUNCT     
_______________________________________________________


In [6]:
def dep_pattern(doc):
    for i in range(len(doc)-1):
        if doc[i].dep_ == 'nsubj' and doc[i+1].pos_ == 'VERB' and doc[i+2].dep_ == 'xcomp':
            for tok in doc[i+2].rights:
                if tok.dep_ == 'obl':
                    return True
    return False

doc1 = nlp(u"Я хочу улететь на Луну.")
doc2 = nlp(u"Мы можем поехать на море.")
doc3 = nlp(u"Я хочу кушать суши 24/7 до конца своих дней.")
doc4 = nlp(u"Солнце зашло 3 часа назад.")
doc5 = nlp(u"Солнце зашло 3 часа назад, небо стало выглядеть очень мрачно ночью.")
doc = [doc1, doc2, doc3, doc4, doc5]

for s in doc:
    print(s)
    if dep_pattern(s):
        print('Found')
    else:
        print('Not found')

Я хочу улететь на Луну.
Found
Мы можем поехать на море.
Found
Я хочу кушать суши 24/7 до конца своих дней.
Found
Солнце зашло 3 часа назад.
Not found
Солнце зашло 3 часа назад, небо стало выглядеть очень мрачно ночью.
Found


#### Использование утилиты Matcher библиотеки spaCy для поиска паттернов последовательностей слов
На самом деле в библиотеке spaCy есть встроенная утилита для решения этой задачи — Matcher, специально разработанная для поиска последовательностей токенов.

In [7]:
doc = nlp(u"Я хочу вкусно покушать сегодня в грузинском ресторане.")
nice_print(doc)

text       pos_       dep_         tag_      
_______________________________________________________
Я          PRON       nsubj        PRON      
хочу       VERB       ROOT         VERB      
вкусно     ADV        advmod       ADV       
покушать   VERB       xcomp        VERB      
сегодня    ADV        advmod       ADV       
в          ADP        case         ADP       
грузинском ADJ        amod         ADJ       
ресторане  NOUN       obl          NOUN      
.          PUNCT      punct        PUNCT     
_______________________________________________________


In [8]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
pattern = [{"DEP": "nsubj"}, {"DEP": "ROOT"}, {"DEP": "advmod"}]
matcher.add("NsubjAuxRoot", [pattern])

matches = matcher(doc)
print(doc)
for match_id, start, end in matches:
        span = doc[start:end]
        print("Span: ", span.text)
        print("The positions in the doc are: ", start, "-", end)


Я хочу вкусно покушать сегодня в грузинском ресторане.
Span:  Я хочу вкусно
The positions in the doc are:  0 - 3


Matcher не позволяет описывать подобные паттерны : «подлежащее + вспомогательный глагол + глагол + ... + прямое дополнение...».

Только четко заданные

#### Применение нескольких паттернов

Одно высказывание можно сопоставить с несколькими паттернами, дабы убедиться, что оно удовлетворяет всем нужным условиям. 
___
! dep - это то, чем является слово в предложении по отношению к другим словам.

!pos - это часть речи. (PRON, VERB)

<b>Замечание</b>: так как для русского языка метод tag_ и pos_ работают одинаково, модифицируем книжную версию под себя.



In [9]:
def dep_pattern(doc):
    for i in range(len(doc)-1):
        if doc[i].dep_ == 'nsubj' and doc[i+1].pos_ == 'VERB' and doc[i+2].dep_ == 'xcomp':
            for tok in doc[i+2].rights:
                if tok.dep_ == 'obl':
                    return True
    return False

def pos_pattern(doc):
    for token in doc:
        if token.dep_ == 'nsubj' and token.tag_ != 'PRON':
            return False
        if token.dep_ == 'ROOT' and token.tag_ != 'VERB':
            return False
        if token.dep_ == 'xcomp' and token.tag_ != 'VERB':
            return False
    return True

doc1 = nlp(u"Я хочу улететь на Луну.")
doc2 = nlp(u"Мы можем поехать на море.")
doc3 = nlp(u"Я хочу кушать суши 24/7 до конца своих дней.")
doc4 = nlp(u"Солнце зашло 3 часа назад.")
doc5 = nlp(u"Солнце зашло 3 часа назад, небо стало выглядеть очень мрачно ночью.")
doc = [doc1, doc2, doc3, doc4, doc5]

# Тестирование кода
for sent in doc:
    print(sent)
    if dep_pattern(sent) and pos_pattern(sent):
        print('Found')
    else:
        print('Not found')

Я хочу улететь на Луну.
Found
Мы можем поехать на море.
Found
Я хочу кушать суши 24/7 до конца своих дней.
Found
Солнце зашло 3 часа назад.
Not found
Солнце зашло 3 часа назад, небо стало выглядеть очень мрачно ночью.
Not found


#### Создание паттернов на основе пользовательских признаков

Например, может понадобиться, чтобы предыдущий сценарий распознавал паттерн, в котором учитывается число местоимений (единственное или множественное).

SpaCy же различает число существительных, но не местоимений.

___
Функция (pron_pattern ) находит все прямые дополнения в передаваемом ей предложении, определяет, является ли
прямое дополнение личным местоимением, а затем выясняет, в каком оно числе: единственном или множественном. 

Далее функция применяется к примеру предложения после проверки его на соответствие двум паттернам,

In [10]:
def pron_pattern(doc):
    plural = ["мы", "нас", "они", "их"]
    for token in doc:
        print(token)
        if token.dep_ == 'nsubj' and token.tag_ == 'PRON':
            if token.text in plural:
                return 'plural'
        else:
            return 'singular'
    return 'not found'

def dep_pattern(doc):
    for i in range(len(doc)-1):
        if doc[i].dep_ == 'nsubj' and doc[i+1].pos_ == 'VERB' and doc[i+2].dep_ == 'obj':
            for tok in doc[i+1].rights:
                if tok.dep_ == 'xcomp':
                    return True
    return False

def pos_pattern(doc):
    for token in doc:
        if token.dep_ == 'nsubj' and token.tag_ != 'PRON':
            return False
        if token.dep_ == 'ROOT' and token.tag_ != 'VERB':
            return False
        if token.dep_ == 'obj' and token.tag_ != 'PRON':
            return False
    return True

doc1 = nlp(u"мы можем их обогнать.")

nice_print(doc1)

if dep_pattern(doc1) and pos_pattern(doc1):
    print('Found:', 'the pronoun in position of direct object is',
    pron_pattern(doc1))
else:
    print('Not found')


doc2 = nlp(u"я могу их обогнать.")

nice_print(doc2)

if dep_pattern(doc2) and pos_pattern(doc2):
    print('Found:', 'the pronoun in position of direct object is',
    pron_pattern(doc2))
else:
    print('Not found')

text       pos_       dep_         tag_      
_______________________________________________________
мы         PRON       nsubj        PRON      
можем      VERB       ROOT         VERB      
их         PRON       obj          PRON      
обогнать   VERB       xcomp        VERB      
.          PUNCT      punct        PUNCT     
_______________________________________________________
мы
Found: the pronoun in position of direct object is plural
text       pos_       dep_         tag_      
_______________________________________________________
я          PRON       nsubj        PRON      
могу       VERB       ROOT         VERB      
их         PRON       obj          PRON      
обогнать   VERB       xcomp        VERB      
.          PUNCT      punct        PUNCT     
_______________________________________________________
я
могу
Found: the pronoun in position of direct object is singular


#### Выбор применяемых паттернов

Столь слабое сцепление между паттернами позволяет использовать их совместно или по отдельности. Например, функцию dep_pattern, проверяющую предложение на соответствие паттерну «подлежа- щее + вспомогательный глагол + глагол + ... + прямое дополнение...», можно использовать вместе с функцией для паттерна «существитель- ное + вспомогательный модальный глагол + глагол в неопределенной форме + ... + существительное...», дабы убедиться, что и подлежащее, и прямое дополнение в предложении — существительные. 


#### Применение паттернов последовательностей слов в чат-ботах для генерации высказываний

В главе 4 мы узнали, как для поддержания разговора с пользователем утвердительное высказывание превратить в соответствующий вопрос. 
С помощью паттернов последовательностей слов можно реагировать на высказывания пользователя и другим образом — например, генерируя подходящие <b>утвердительные высказывания</b>.

___


Что хотим от чат бота???

- Символы четко различимы. Я могу быстро их распознать.

- Я тоже могу быстро распознавать символы.

In [11]:
Image(url="images/picture_9.png", width=450, height=400)

вместо "too" используем "тоже"

In [12]:
# находит все прямые дополнения в передаваемом ей предложении, определяет, является ли прямое дополнение
# личным местоимением, а затем выясняет, в каком оно числе: единственном или множественном
def pron_pattern(doc):
    plural = ["мы", "нас", "они", "их"]
    for token in doc:
        if token.dep_ == 'nsubj' and token.tag_ == 'PRON':
            if token.text in plural:
                return 'plural'
        else:
            return 'singular'
    return 'не найдено'

# реализует есть ли в doc заданный паттерн
def dep_pattern(doc):
    for i in range(len(doc)-1):
        if doc[i].dep_ == 'nsubj' and doc[i+1].pos_ == 'VERB' and doc[i+2].pos_ == 'ADV':
            for tok in doc[i+1].rights:
                if tok.dep_ == 'xcomp':
                    return True
    return False

# реализует есть ли в doc заданный паттерн (из-за неполных методов для русского языка работает почти также как dep_pattern)
def pos_pattern(doc):
    for token in doc:
        if token.dep_ == 'nsubj' and (token.tag_ != 'PRON' and token.tag_ != 'NOUN'):
            return False
        if token.dep_ == 'ROOT' and (token.tag_ != 'VERB' and token.tag_ != 'ADJ'):
            return False
    return True

# поиск существительного, соответствующего личному местоимению (tag_ работает не так как нужно и равен dep_)
def find_noun(sents, num):
    if num == 'plural':
        for sent in reversed(sents):
            for token in sent:
                if token.dep_ == "nsubj" and token.text == "Символы":
                    return token.text
    if num == 'singular':
        for sent in reversed(sents):
            for token in sent:
                if token.dep_ == "nsubj":
                    return token.text  

    return 'Существительное не найдено'

# генерирует код для анализа высказывания
def gen_utterance(doc, noun):
    sent = ''
    for i, token in enumerate(doc):
        if token.dep_ == 'obj' and token.tag_ == 'PRON':
            sent = doc[0:1].text + ' тоже ' + doc[1:i].text + ' ' + doc[i+1:len(doc)-1].text + ' ' + noun.lower() + '.'
            # sent = doc[:i].text + ' ' + noun.lower() + ' ' + doc[i+1:len(doc)-1].text + ' тоже.'
            return sent
    return 'Не удалось сгенерировать высказываниe'


Тестирование верхних функций

In [13]:
doc = nlp(u'Символы четко различимы. Я могу быстро их распознать.')
sents = list(doc.sents)

for sent in sents:
    print(sent.text, sent.start_char, sent.end_char, sent.label_)  
displacy.render(doc, style = 'dep')

Символы четко различимы. 0 24 
Я могу быстро их распознать. 25 53 


In [14]:
for i, sent in enumerate(sents):
    if dep_pattern(sent) and pos_pattern(sent):
        noun = find_noun(sents[:i], pron_pattern(sent))
        if noun != 'Существительное не найдено':
            response = gen_utterance(sents[i], noun)
            break

print("USER: ", doc)
print("BOT: ", response)

USER:  Символы четко различимы. Я могу быстро их распознать.
BOT:  Я тоже могу быстро распознать символы.


Еще один пример

In [15]:
doc = nlp(u'Персики очень вкусные. Я могу быстро их съесть.')
sents = list(doc.sents)

for i, sent in enumerate(sents):
    if dep_pattern(sent) and pos_pattern(sent):
        noun = find_noun(sents[:i], pron_pattern(sent))
        if noun != 'Существительное не найдено':
            response = gen_utterance(sents[i], noun)
            break

print("USER: ", doc)
print("BOT: ", response)

USER:  Персики очень вкусные. Я могу быстро их съесть.
BOT:  Я тоже могу быстро съесть персики.


#### <b>Выделение ключевых слов из деревьев синтаксических зависимостей</b>
C задачей выделения смысла текста такие паттерны помогут не всегда. Допустим, есть фраза:
- Мне нужен авиабилет до Берлина.

Пункт назначения - именнованая сущность GPE, однако, что если будут стоять фразы:
- Я еду на конференцию в Берлин. Мне нужен авиабилет.
- Я еду на конференцию, которая пройдет в Берлине. Я хотел бы забронировать авиабилет.

Или такая ситуация:
- Я хочу забронировать билет на прямой рейс без посадки в Берлине.

*Для получения необходимой информации часто приходится исследовать отношения слов в предложении. Именно для этого может пригодиться обход дерева зависимостей предложения.*

<b>Суть в том:</b> чтобы обойти только часть, и остановиться сразу же после результата.


#### <b> Выделение информации путем обхода дерева зависимостей</b>

In [16]:
doc = nlp(u"Мне нужен авиабилет до Берлина.")
nice_print(doc)
displacy.render(doc, style='dep', jupyter=True)

text       pos_       dep_         tag_      
_______________________________________________________
Мне        PRON       iobj         PRON      
нужен      ADJ        ROOT         ADJ       
авиабилет  NOUN       nsubj        NOUN      
до         ADP        case         ADP       
Берлина    PROPN      nmod         PROPN     
.          PUNCT      punct        PUNCT     
_______________________________________________________


Видно, что корневое слово - это 'нужен', то есть он не является дочерним ни для какого другого слова.

Например, если пройти по всем дочерним элементам от корневого элемента справа, то можно добраться до Берлина

#### <b> Проход в цикле по главным элементам токенов</b>

Один из способов добраться до Берлина - это, как было сказано выше, идти по правосторонним дочерним элементам всех токенов.

Если подобным образом можно пройти от to до Berlin, значит, можно обоснованно считать, что между этими двумя словами существует семантическая связь. Но у этого способа есть недостаток:

- Я еду на конференцию по Spacy, которая пройдет в Берлине.


In [17]:
doc = nlp(u"Я еду на конференцию по Spacy, которая пройдет в Берлине")

displacy.render(doc, style='dep', jupyter=True)

In [18]:
doc = nlp(u"Я хочу заказать пиццу")
displacy.render(doc, style='dep', jupyter=True)

Видно, что у слова *конференция* есть два непосредственных правых дочерних элемента, поэтому приходится просматривать несколько веток дерева, что усложняет код.

С другой стороны, хотя у главного элемента может быть несколько дочерних, у каждого слова в предложении ровно один главный элемент. Это значит, что можно двигаться наоборот, справа налево, начав со слова Berlin, и пытаться достичь слова to.

Реализуем это в функции!

In [19]:
def det_destination(doc):
    for i, token in enumerate(doc):
        if token.ent_type != 0 and token.ent_type_ == 'LOC':
            while True:
                token = token.head
                if token.text == 'конференцию':
                    return doc[i].text
                if token.head == token:
                    return 'Не удалось определить'
    return 'Не удалось определить'


doc = nlp(u'Я еду на конференцию в Берлин.')
displacy.render(doc, style='dep', jupyter=True)
for token in doc:
    print(token.ent_type, "   ", token.ent_type_)
    
# Тестирование функции det_destination    
dest = det_destination(doc)
print('Похоже, пользователь хочет получить билет до: ' + dest)



0     
0     
0     
0     
0     
385     LOC
0     
Похоже, пользователь хочет получить билет до: Берлин


#### <b> Краткое изложение текста с помощью деревьев зависимостей </b>

Конечно, сфера применения подхода с деревьями синтаксических зависимостей не ограничивается чат-ботами. Этот подход можно использовать, например, в приложениях, предназначенных для обработки отчетов. Представьте, что вам нужно разработать приложение для краткого изложения отчетов о розничных продажах, чтобы в них была лишь самая важная информация.




In [20]:
doc = nlp(u"В первом квартале продажи продукта достигли нового рекорда - было продано 18,6 миллиона единиц.")
# хотим - Продажи продукта достигли 18,6 миллиона единиц.
displacy.render(doc, style='dep', jupyter=True)

In [21]:
nice_print(doc)

text       pos_       dep_         tag_      
_______________________________________________________
В          ADP        case         ADP       
первом     ADJ        amod         ADJ       
квартале   NOUN       obl          NOUN      
продажи    NOUN       nsubj        NOUN      
продукта   NOUN       nmod         NOUN      
достигли   VERB       ROOT         VERB      
нового     ADJ        amod         ADJ       
рекорда    NOUN       obj          NOUN      
-          NOUN       obl          NOUN      
было       AUX        aux:pass     AUX       
продано    VERB       conj         VERB      
18,6       NUM        nummod       NUM       
миллиона   NOUN       nummod:gov   NOUN      
единиц     NOUN       nsubj:pass   NOUN      
.          PUNCT      punct        PUNCT     
_______________________________________________________


In [22]:
phrase = ''
for token in doc:
    if token.dep_.find('nummod') != -1:
        if phrase.find(token.text) == -1:
            phrase = phrase + ' ' + token.text
        while True:
            if token.head and phrase.find(token.head.text) == -1:
                phrase = phrase + ' ' + token.head.text
                break
            elif list(token.rights) != []:
                phrase = phrase + ' ' + token.rights
                break
            elif list(token.lefts) != []:
                phrase = phrase + ' ' + token.lefts
                break
            break
    


print(phrase.strip())

18,6 миллиона единиц


In [23]:
while True:
    token = doc[token.i].head
    if token.pos_ != 'ADP': 
        phrase = token.text + phrase
    if token.dep_ == 'ROOT':
        break

print(phrase)

достигли 18,6 миллиона единиц


In [24]:
for tok in token.lefts:
    if tok.dep_ == 'nsubj':
        phrase = tok.text + ' ' + ' '.join([tok.text for tok in tok.rights]) + ' ' + phrase
        break
print(phrase)

продажи продукта достигли 18,6 миллиона единиц


Этот результат — сжатая версия исходного предложения.

In [25]:
Image(url="images/picture_10.png", width=450, height=400)

In [26]:
def main_sentence(doc):
    phrase = ''
    for token in doc:
        if token.dep_.find('nummod') != -1:
            if phrase.find(token.text) == -1:
                phrase = phrase + ' ' + token.text
            while True:
                if token.head and phrase.find(token.head.text) == -1:
                    phrase = phrase + ' ' + token.head.text
                    break
                elif list(token.rights) != []:
                    phrase = phrase + ' ' + token.rights
                    break
                elif list(token.lefts) != []:
                    phrase = phrase + ' ' + token.lefts
                    break
                break
    while True:
        token = doc[token.i].head
        if token.pos_ != 'ADP': 
            
            phrase = token.text + phrase
        if token.dep_ == 'ROOT':
            break

    for tok in token.lefts:
        if tok.dep_ == 'nsubj':
            phrase = tok.text + ' ' + ' '.join([tok.text for tok in tok.rights]) + ' ' + phrase
            break
        if len(list(token.lefts)) == 1:
            while True:
                token = list(token.lefts)[0]
                for tok in token.lefts:
                    if tok.dep_ == 'nsubj':
                        phrase = tok.text + ' ' + ' '.join([tok.text for tok in tok.rights]) + ' ' + phrase
                        break
                break
    return phrase


In [27]:
# Хотим - "Выручка компании составила 4,26 миллиона долларов."
doc = nlp(u"Компания, чья прибыль достигла рекордно высокого уровня в этом году, во многом благодаря изменениям в руководстве, заработала общий доход в размере 4,26 миллиона долларов.")

displacy.render(doc, style='dep', jupyter=True)

In [28]:
print(main_sentence(doc))

Компания достигла заработала 4,26 миллиона долларов


#### <b>Усовершенствование чат-бота для бронирования билетов с помощью учета контекста </b>

Чтобы отдача от этих сценариев была больше, необходимо учитывать контекст для выбора подходящего ответа. 

Расширим функциональность сценария бронирования билетов, чтобы он мог обрабатывать более широкий спектр вводимых пользователями данных, включая высказывания, не содержащие пару to + GPE в каком-либо сочетании. 

Рассмотрим следующее высказывание:
- Я присутствую на конференции в Берлине

В подобных случаях чат-бот может задать подтверждающий вопрос наподобие такого:
- Ты ведь хочешь билет до Берлина, верно?

Этот усовершенствованный чат-бот для бронирования билетов реагирует по-разному в трех различных ситуациях.
1. Пользователь выражает ясное намерение забронировать билет в определенное место.
2. Сразу не понятно, хочет пользователь забронировать билет в упомя- нутый им пункт назначения или нет.
3. Текст пользователя не содержит упоминания о каком-либо пункте назначения.

In [29]:
Image(url="images/picture_11.png", width=450, height=250)

In [30]:
# Функция для определения пункта назначения, 1 тип ввода
def det_destination(doc):
    for i, token in enumerate(doc):
        if token.ent_type != 0 and token.ent_type_ == 'LOC':
            while True:
                token = token.head
                if token.text == 'конференцию':
                    return doc[i].text
                if token.head == token:
                    return 'Failed to determine'
    return 'Failed to determine'

# обнаружение локации, 2 тип ввода
def guess_destination(doc):
    for token in doc:
        if token.ent_type != 0 and token.ent_type_ == 'LOC':
            return token.text
    return 'Failed to determine'

def gen_response(doc):
    dest = det_destination(doc)
    if dest != 'Failed to determine':
        return 'Когда вам нужно быть в ' + dest + '?'
    dest = guess_destination(doc)
    if dest != 'Failed to determine':
        return 'Тебе нужен билет до ' + dest +', верно?'
    return 'Вы куда-то летите?'

In [31]:
doc = nlp(u'Я еду на конференцию в Берлин.')
print(gen_response(doc))

doc = nlp(u'Я присутствую на конференции в Берлине.')
print(gen_response(doc))

doc = nlp(u'Я устала.')
print(gen_response(doc))

Когда вам нужно быть в Берлин?
Тебе нужен билет до Берлине, верно?
Вы куда-то летите?


#### <b>Повышаем IQ чат-бота за счет поиска подходящих модификаторов</b>
Чтобы сделать чат-бот умнее, можно воспользоваться деревьями зави- симостей для поиска модификаторов для конкретных слов. Например, научив наше приложение распознавать прилагательные, подходящие для заданного существительного, достаточно будет просто сказать чат-боту: 

- Я хотела бы почитать книгу.
- Хотите художественную книгу?

<b>Модификатор (modifier)</b> — необязательный элемент фразы или про- стого предложения, меняющий смысл другого элемента. Удаление модификатора не всегда меняет основной смысл предложения, но делает его более общим. 


В качестве небольшого примера рассмотрим следующие два предложения:

- Я хочу почитать книгу.
- Я хочу прочитать книгу по Python.

Здесь конкретизация идет во втором предложении "по Python" - модификатор.


In [32]:
doc = nlp(u"Этот экзотический фрукт из Африки.")
displacy.render(doc, style='dep', jupyter=True)

<b>премодификаторы (premodifiers)</b> - фрукт, Этот и экзотический

<b>постмодификатор (postmodifier)</b> - из Африки

Пусть нам нужно установить возможные прилагательные модификаторы для слова **фрукт**. (Прилагательные-модификаторы — это всегда премодификаторы.) Кроме того, хотелось бы знать, какие сущности типа **LOC** можно найти в числе постмодификаторов того же слова. Данная информация в дальнейшем окажется полезной при генерации высказываний во время беседы о фруктах.

In [33]:
doc = nlp(u"Мякоть кивано имеет желеобразную консистенцию с освежающим фруктовым вкусом. Это приятный экзотический фрукт из Африки. Это определенно стоит попробовать.")
displacy.render(doc, style='dep', jupyter=True)

fruit_adjectives = []
fruit_origins = []

for token in doc:
    if token.text == 'фрукт':
        fruit_adjectives = fruit_adjectives + [modifier.text for modifier in token.lefts if modifier.pos_ == 'ADJ']
        fruit_origins = fruit_origins + [modifier.text for modifier in token.rights if modifier.ent_type_ == 'LOC']
        
print('Список модификаторов прилагательных к слову фрукт:', fruit_adjectives)
print('Список имен LOC, применимых к слову фрукт в качестве постмодификаторов:', fruit_origins)

Список модификаторов прилагательных к слову фрукт: ['приятный', 'экзотический']
Список имен LOC, применимых к слову фрукт в качестве постмодификаторов: ['Африки']


#### **Резюме**

Паттерны на основе лингвистических признаков хорошо подходят для распознавания общей структуры предложений, имеющих подлежащее, вспомогательный модальный глагол, смысловой глагол и прямое дополнение.

Но на практике в приложениях приходится распознавать более сложные варианты структур и обрабатывать более широкий спектр текстовых данных, вводимых пользователями. Здесь пригодятся **деревья зависимостей предложений**.